## Pipeline AMI Set-Up

before using new key, remember to chmod so not public:

    chmod 0400 ~/Keys/abirmingham_oregon.pem 

ssh into an empty Amazon Linux instance:

    ssh -i ~/Keys/abirmingham_oregon.pem ec2-user@XX.XX.XXX.XXX
    screen
    
    curl https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -o miniconda_py3.sh
    md5sum miniconda_py3.sh 
    
compare to http://repo.continuum.io/miniconda/ value

    bash miniconda_py3.sh
    logout
    
ssh back in again to get new terminal, then verify conda now in path: 

    echo $PATH
    
    conda update conda
    conda --version
    
conda 4.1.4

    conda create -n crispr_pipeline python=3.4 git matplotlib pandas jupyter
    source activate crispr_pipeline
    
Note that it is important to force python 3.4, which the pipeline was developed on, as the miniconda default is 3.5, which is apparently different enough to break things somewhere in nbformat ....  Install cutadapt, which isn't available as part of the main conda "channel" but is available through conda from bioconda:

    conda install -c bioconda cutadapt  

Now ensure I'm using the conda pip:

    which pip 

    pip install nbparameterise 
    
have to do this because nbparameterise isn't yet available as a conda package

    conda list
    
successfully shows both the stuff (git, pandas, jupyter) installed with conda and the stuff (nbparameterise) installed with conda's pip.

    git clone https://github.com/ucsd-ccbb/jupyter-genomics.git
    
Ok, now there's a jupyter-genomics folder in the directory I called this from (`/home/ec2-user/`).

    mkdir library_definitions
    cd library_definitions
    
    aws configure

Ok, **NB** that awscli on this image is now set up with MY credentials ...

    # aws s3 cp --recursive s3://ccbb-analysis/Amanda/temporary/ . 
    aws s3 cp s3://ccbb-analysis/Amanda/temporary/CV4_2spacers.txt CV4_2spacers.txt
    aws s3 cp s3://ccbb-analysis/Amanda/temporary/Metabolism_dual_spacers.txt Metabolism_dual_spacers.txt
    
Now also **NB** that the image now includes Mali's (presumably unpublished) library design files.

    source deactivate
    exit
    logout

## Per-Instance Configuration

My new AMI has id `ami-d1b37fb1`.

Start a new instance of this with an EBS drive, and ssh in:

    ssh -i ~/Keys/abirmingham_oregon.pem ec2-user@XX.XX.XXX.XXX
    screen
    
put file system on new drive, mount, and assign to my working user:

    lsblk
    sudo file -s /dev/xvdb
    sudo mkfs -t ext4 /dev/xvdb
    sudo mkdir /data
    sudo mount /dev/xvdb /data
    sudo chown -R ec2-user /data
    
set up the expected locations:

    cd /data
    mkdir raw
    mkdir interim
    mkdir processed

## Per-Run Configuration

If starting a previously stopped instance, login and mount the drive:

    ssh -i ~/Keys/abirmingham_oregon.pem ec2-user@XX.XX.XXX.XXX
    screen
    sudo mount /dev/xvdb /data

download the raw data to the expected location:

    cd /data/raw
    mkdir 160817_D00611_0339_BHWTT2BCXX
    cd 160817_D00611_0339_BHWTT2BCXX    
    wget --user mali --password XXXXX -nd ftp://igm-storage1.ucsd.edu/160817_D00611_0339_BHWTT2BCXX/Data/Fastq/*.fastq.gz
    
get the latest version of the jupyter genomics code:

    cd ~/jupyter-genomics/
    source activate crispr_pipeline
    git pull

Switch to the directory where the pipeline code lives and run the pipeline pre-configured for the AWS file structure:

    cd ~/jupyter-genomics/src/crispr/
    source activate crispr_pipeline
    
Open the pipeline script and modify the necessary variables at the top:

    vim mali_pipeliner_aws.py
    
Now run the script:    
    
    python mali_pipeliner_aws.py
    
After the run is complete, zip the resulting directory and upload it to s3:

    cd /data/processed/
    [find name of output dir]
    zip -r 20160801_LN229_CV4_19mer_1mm_py_20160804205646.zip 20160801_LN229_CV4_19mer_1mm_py_20160804205646/
    aws s3 cp 20160801_LN229_CV4_19mer_1mm_py_20160804205646.zip s3://ccbb-analysis/Amanda/20160210_mali_crispr/ 
    
    # aws s3 cp --recursive . s3://ccbb-analysis/Amanda/20160210_mali_crispr/ 

## Set-Up Through Run Steps On Mac:

    bash Miniconda3-latest-MacOSX-x86_64.sh
    
close terminal, open new terminal, verify conda now in path: 

    echo $PATH

        /Users/Birmingham/Applications/miniconda3/bin:/usr/local/bin:/usr/local/sbin:/usr/bin:/bin:/usr/sbin:/sbin
    
    conda update conda
    conda --version

        conda 4.1.8
        
    conda create -n crispr_pipeline python=3.4 git matplotlib pandas jupyter
    source activate crispr_pipeline
    conda install -c bioconda cutadapt 
    pip install nbparameterise
    
    curl -u mali:iVO5NTQ ftp://igm-storage1.ucsd.edu/160706_D00611_0304_BHVVJ3BCXX/Data/Fastq/A549_MV4_d14_1_S3_L001_R2_001.fastq.gz -o A549_MV4_d14_1_S3_L001_R2_001.fastq.gz
    
    cd ~/Repositories/jupyter-genomics/src/crispr/
    python mali_pipeliner_aws_local.py
    
